# Train model - No HyperDrive

In [ ]:
!pip install azureml-sdk[notebooks]
!pip install torch
!pip install torchvision
!pip install tensorflow

In [ ]:
# Imports
import torch
import torchvision
import string
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
# Azure ML Imports
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails

## Workspace

In [ ]:
ws = Workspace.from_config()
experiment_name = 'ASL-DeepLearning-hyperameter'

exp_with_hyper = Experiment(workspace=ws, name='Sign-Language-HyperDrive')
exp_no_hyper = Experiment(workspace=ws, name='Sign-Language-NoHyperDrive')


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


## Compute

In [ ]:
# Choose a name for your CPU cluster
cluster_name = "gpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC24',
                                                           max_nodes=10)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://www.kaggle.com/datamunge/sign-language-mnist"

found = False
key = "sign-language-mnist"
description_text = "sign Language MNIST"

if key in ws.datasets.keys(): 
    found = True
    ds = ws.datasets[key] 

if not found:
    from azureml.data.dataset_factory import TabularDatasetFactory
    datastore_path = "https://github.com/emanbuc/ASL-Recognition-Deep-Learning/raw/main/datasets/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv"
    ds = TabularDatasetFactory.from_delimited_files(path=datastore_path,header=True)       
    #Register Dataset in Workspace
    ds = ds.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = ds.to_pandas_dataframe()
df.describe()

In [ ]:
# To map each label number to its corresponding letter
letters = dict(enumerate(string.ascii_uppercase))
letters

## Data Preparation

In [ ]:
targets = pd.get_dummies(df.pop('label')).values
data = df.values
targets.shape

In [ ]:
data.shape

In [ ]:
from sklearn.preprocessing import minmax_scale
data = minmax_scale(data)

In [ ]:
input_shape = (28,28, 1) # 28*28 = 784

In [ ]:
data = np.reshape(data,(-1, 28, 28,1))
data.shape

In [ ]:
targets.shape

## SPLITTING THE DATA Training set and Validation Set

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(data, targets , test_size = 0.2, random_state=0)

### Save data to file for remote cluster

Salva i dati prepatati su file con pickle e esegue upload verso datastore del workspace in cloud.

In [ ]:
import pickle
import os
if os.path.isfile('dataset/sign-language-mnist.pkl'):
    print('File is present')
else:
    os.makedirs('dataset')
    with open('dataset/sign-language-mnist.pkl','wb') as f:
        pickle.dump((X_train,X_val,y_train,y_val),f)
    
    datastore=ws.get_default_datastore()
    datastore.upload('./dataset', target_path='sign-language-mnist-data')

print('Done')

## Remote Trainig Experiment

Create an estimator object to run training experiment in remote compute cluster

In [ ]:
from azureml.train.estimator import Estimator
script_params = {
    '--data_folder': ws.get_default_datastore(),
    '--hidden': 100
}

est = Estimator(source_directory='.',
                script_params=script_params,
                compute_target=cpu_cluster,
                entry_script='train_keras.py',
                pip_packages=['keras','tensorflow'])

In [ ]:

run = exp_no_hyper.submit(est)

## Remote Training with Hyper Drive

### Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
from azureml.train.hyperdrive.policy import BanditPolicy,MedianStoppingPolicy
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal,choice

In [ ]:
#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
         '--hidden': choice([50,100,200,300]),
         '--batch_size': choice([64,128]), 
         '--epochs': choice([3,5,10]),
         '--dropout': choice([0.5,0.8,1])})

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.

early_termination_policy = MedianStoppingPolicy()
hd_config = HyperDriveConfig(estimator=est,
  hyperparameter_sampling=param_sampling,
  policy=early_termination_policy,
  primary_metric_name='Accuracy',
  primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
  max_total_runs=16,
  max_concurrent_runs=4)

param_sampling = RandomParameterSampling({
         '--hidden': choice([50,100,200,300]),
         '--batch_size': choice([64,128]), 
         '--epochs': choice([5,10,50]),
         '--dropout': choice([0.5,0.8,1])})

In [ ]:
hyperdrive_run = exp_with_hyper.submit(hd_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
assert(hyperdrive_run.get_status() == "Completed")
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
best_run_metrics = best_run.get_metrics()
print('Best accuracy: {}'.format(best_run_metrics['Accuracy']))
print('========================')

In [ ]:
model = best_run.register_model(model_name='mnist_model.hdf5', model_path='./outputs/mnist_model.hdf5')

In [ ]:
best_run_metrics

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

Deploy your model as a web service using Model.deploy(). Web services take one or more models, load them in an environment, and run them on one of several supported deployment targets.  For this example, we will deploy your scikit-learn model to an Azure Container Instance (ACI).

In [ ]:
service_name = 'sign-language-service'

service = Model.deploy(ws, service_name, [model], overwrite=True)
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

## Test Deployed Model

In [1]:
import pickle
import pandas as pd
import numpy as np

test_data = pd.read_csv("../datasets/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")
print("test_data: "+ str(test_data.shape))
X_test=test_data.iloc[1:10,1:785]
print("X_test: " +str(X_test.shape))

test_data: (7172, 785)
X_test: (9, 784)


In [2]:
testdict= X_test.to_dict(orient="index")

In [4]:
# Two sets of data to score, so we get two results back
data = {"data": (testdict[1],testdict[2],testdict[3],testdict[4],testdict[5],testdict[5])}

In [5]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = 'http://fa662e2e-8a0a-419c-8258-af98c3380389.southcentralus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = 'brhKng8SDm6C8QLQRuH7VqOsetHebyLC'

# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.get_logs()

In [ ]:
service.delete()

In [ ]:
#cpu_cluster.delete()